# General Code

In [1]:
# imports
import time
import math
import random
from enum import Enum
import webbrowser
import ipywidgets as widgets
import urllib
import numpy as np
import pyaudio
from IPython.display import display, HTML

In [2]:
# setup style for embedded HTML output / ipywidgets
display(HTML(
'<style>'\
   '.cell-output-ipywidget-background { background-color: transparent !important; }'\
   '.jp-OutputArea-output             { background-color: transparent; }'\
   'h1 { display: inline; font-size: 60px; font-family: Serif; }'\
'</style>'))

In [3]:
# definitions/lists
class Patterns(Enum):
    major = 1
    natural_minor = 2

notes           = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
note_names      = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']
keys            = [['C',0], ['G',1], ['D',2], ['A',3], ['E',4], ['B',5], ['F#/Gb',6], ['Db',-5], ['Ab',-4], ['Eb',-3], ['Bb', -2], ['F',-1]]
intervals       = ['P1', 'm2/m9', 'M2/M9', 'm3', 'M3', 'P4/P11', 'a4/d5', 'P5', 'a5/m6/m13', 'M6/M13/bb7', 'm7', 'M7']
interval_ratios = [1/1, 16/15, 9/8, 6/5, 5/4, 4/3, 45/32, 3/2, 8/5, 5/3, 9/5, 15/8, 2/1]
modes           = ['Ionian', 'Dorian', 'Phrygian', 'Lydian', 'Mixolydian', 'Aeolian', 'Locrian']
chords          = [
    ['',       ['P1','M3','P5']],
    ['m',      ['P1','m3','P5']],
    ['dim',    ['P1','m3','d5']],
    ['aug',    ['P1','M3','a5']],
    ['sus2',   ['P1','M2','P5']],
    ['sus4',   ['P1','P4','P5']],
    ['sus',    ['P1','P4','P5']],
    ['m6',     ['P1','m3','P5','M6']],
    ['6',      ['P1','M3','P5','M6']],
    ['dim7',   ['P1','m3','d5','bb7']],
    ['m7b5',   ['P1','m3','d5','m7']],
    ['m7',     ['P1','m3','P5','m7']],
    ['7',      ['P1','M3','P5','m7']],
    ['maj7',   ['P1','M3','P5','M7']],
    ['m(add9)',['P1','m3','P5','M9']],
    ['add9',   ['P1','M3','P5','M9']]
]

major_scale         = [  1,  2,  3,  4,  5,  6,  7,  8 ]
natural_minor_scale = [  1,  2, -3,  4,  5, -6, -7,  8 ]
major_intervals     = [  0,  2,  2,  1,  2,  2,  2,  1 ]

practices = [
    'scales/intervals on guitar',
    'scales/intervals on keyboard',
]

In [4]:
# note functions
def relativeToneLetter(tone, offset):
    """Return the simple offset from a tone name and integer offset
       i.e., relativeToneLetter('cb',-3) ==> 'G'
    """
    width = ord('G') - ord('A') + 1
    i = ord(tone[0].upper()) + offset
    if (i < ord('A')): i += width
    if (i > ord('G')): i -= width
    return chr(i)

def fullToneName(tone):
    """Return the full tone name from a simple entry
       i.e., fullToneName('bb') ==> 'A#/Bb'
    """
    if len(tone) < 2: return tone.upper()
    tone_name = tone[0].upper() + tone[-1:]
    if (tone_name[1] == 'b'):
        tone_name = relativeToneLetter(tone_name[0],-1) + '#/' + tone_name
    elif (tone_name[1] == '#'):
        tone_name = tone_name + '/' + relativeToneLetter(tone_name[0],1) + 'b'
    try:
        i = notes.index(tone_name)
        return tone_name
    except ValueError:
        return 'X'

def adjustScale(scale):
    """Take a scale as an array of strings, and select any split note names
       like "C#/Db" to create a well-ordered scale
    """
    root = scale[0]
    if root == 'F' or (len(root) > 1 and root[1] == 'b'):
        flat = True
    else:
        flat = False
    letter = root[0]
    new_scale = []
    for i, tone in enumerate(scale):
        if i == 0:
            new_scale.append(root)
        else:
            note = tone
            if tone[0] != relativeToneLetter(letter,1):
                if len(tone) == 1:
                    if tone[0] == letter:
                        note = relativeToneLetter(letter,1) + 'b'
                    else:
                        note = relativeToneLetter(letter,1) + '#'
                else:
                    note = tone.split('/')[1]
            elif len(tone) > 2:
                note = tone.split('/')[1 if flat else 0]
            letter = note[0]
            new_scale.append(note)
    return new_scale

def getScale(root, pattern=Patterns.major):
    """Return a scale as an array of strings for a given root note and a pattern
       designation like Patterns.major. The major diatonic scale will be returned
       if the pattern argument is left out.
    """
    scale = []
    tone = notes.index(fullToneName(root))

    scale_pattern = major_scale
    if (pattern == Patterns.natural_minor):
        scale_pattern = natural_minor_scale

    for degree in scale_pattern:
        tone = (tone + major_intervals[abs(degree)-1]) % len(notes)
        scale.append(notes[ (tone-1) if degree < 0 else tone ])

    scale[0] = root.upper()
    if len(scale[0]) > 1 and scale[0][1] == 'B':
        scale[0] = scale[0][0] + 'b'

    return adjustScale(scale)

def noteIndex(note):
    """Return the index into the notes[] array for the given note name
       i.e., noteIndex('bb') ==> 10
    """
    if len(note) == 1:
        return notes.index(note.upper())
    else:
        a = note[0].upper() + note[1]
        for i in range(len(notes)):
            if notes[i].find(a) != -1:
                return i
    return -1

def getInterval(root, note):
    """Return the musical interval between the given root and target notes
       i.e., getInterval('bb','g#') ==> 'm7'
    """
    ir = noteIndex(root)
    iv = noteIndex(note)
    d = ((iv - ir) + (12 if ((iv - ir) < 0) else 0)) % 12
    return intervals[d]

In [5]:
# frequency functions
def noteToFreq(note, octave):
    """Returns the frequency of a note in Hz
       i.e., the freq of middle C would be noteToFreq('C',4) ==> 261.625...
    """
    x = 12*(octave-4)+notes.index(note)-9
    return 440.*(2**(1./12.))**x

def freqToNote(freq_in):
    """Return the closest note letter, octave, and deviation from center (in cent)
       to a given frequency
       i.e., freqToNote(261.3) ==> ('C', 4, -2.1556...)
    """
    x = math.log(freq_in / 440., 2**(1./12.))
    octave = int(4 + (x + 9)/12)
    note_a = int((x + 9) % 12)

    f = noteToFreq(notes[note_a],octave)
    cent = 1200.*math.log2(freq_in/f)
    if abs(cent) > 50:
        note_a = (note_a + 1) if note_a < 11 else 0
        octave += 1 if note_a == 0 else 0
        f = noteToFreq(notes[note_a],octave)
        cent = 1200.*math.log2(freq_in/f)

    return(notes[note_a], octave, cent)

def getIntervalRatio(interval):
    """Return the frequency ratio from an interval name
       i.e., getIntervalRatio('d5') ==> 1.40625 (=45/32)
    """
    for i in range(len(intervals)):
        if intervals[i].find(interval) != -1:
            break
    interval_number = int(interval.replace('P','').replace('m','').replace('M','').replace('a','').replace('d','').replace('b',''))
    if interval_number > 7:
        return 2*interval_ratios[i]
    return interval_ratios[i]

In [51]:
# sound functions
volume = 0.5                # range [0.0, 1.0]
fs = 44100                  # sampling rate, Hz, must be integer
fadeOut = int(fs * 0.05)    # fadeout waves for 0.1sec

pyAudio = pyaudio.PyAudio()
paStream = pyAudio.open(format=pyaudio.paFloat32, channels=1, rate=fs, output=True)

def playWave(samples):
    global volume, paStream
    bytes = (volume * samples).tobytes()
    paStream.write(bytes)

def createWave(f,duration):
    """Return a sine wave at a frequency for a duration as an np.float32 array
    fade out the wave to keep it from popping
    """
    global fs, fadeOut
    wave = (np.sin(2 * np.pi * np.arange(fs * duration) * f / fs)).astype(np.float32)
    for i in range(fadeOut):
        wave[-i] *= i/fadeOut
    return wave

def createChordWave(f_root,intervals,duration):
    sample = 0*createWave(1,duration)
    for tone in intervals:
        f = f_root * getIntervalRatio(tone)
        sample = np.add(sample,createWave(f,duration))
    return sample / np.max(sample)

def createNoteWaves(f_root,intervals,duration):
    noteSamples = []
    for note in intervals:
        f = f_root * getIntervalRatio(note)
        noteSamples.append(createWave(f,duration))
    return noteSamples

# Examples

In [7]:
# example: Get a random key and its natural minor scale
key = random.randint(0, len(keys)-1)
key_name = keys[key][0]
scale = getScale(key_name,pattern=Patterns.natural_minor)

print(f'{key_name} : {scale}')

B : ['B', 'C#', 'D', 'E', 'F#', 'G', 'A', 'B']


In [8]:
# example: What notes are in the harmonic series?
C4 = noteToFreq('C',4)
print('Harmonic series of middle C:')
print(f' Harm   Freq[Hz]    Note   Err    Int')
for i in range(15):
    h = i+1
    f = C4*h
    n,o,c = freqToNote(f)
    iv = getInterval('C',n)
    print(f'  {h:2d}    {f:7.2f}   {n:>5s}{o}   {c:+3.0f}   {iv:>5s}')

print('...')

Harmonic series of middle C:
 Harm   Freq[Hz]    Note   Err    Int
   1     261.63       C4    +0      P1
   2     523.25       C5    -0      P1
   3     784.88       G5    +2      P5
   4    1046.50       C6    -0      P1
   5    1308.13       E6   -14      M3
   6    1569.75       G6    +2      P5
   7    1831.38   A#/Bb6   -31      m7
   8    2093.00       C7    -0      P1
   9    2354.63       D7    +4   M2/M9
  10    2616.26       E7   -14      M3
  11    2877.88   F#/Gb7   -49   a4/d5
  12    3139.51       G7    +2      P5
  13    3401.13   G#/Ab7   +41   a5/m6/m13
  14    3662.76   A#/Bb7   -31      m7
  15    3924.38       B7   -12      M7
...


In [36]:
# example: Get a random modal key and search for a backing track
key = random.randint(0, len(note_names)-1)
mode = random.randint(0, len(modes)-1)

modal_key = note_names[key] + " " + modes[mode]

url = "https://www.youtube.com/results?search_query=backing+track+" + urllib.parse.quote(note_names[key]) + "+" + modes[mode]

button = widgets.Button(description='Find a backing track')
button.on_click(lambda b: webbrowser.open(url))

display(HTML(f'<h1>{modal_key}</h1>'), button)

Button(description='Find a backing track', style=ButtonStyle())

In [67]:
# example: Get a random chord
note = random.randint(0, len(note_names)-1)
note_name = note_names[note]

chord_number = random.randint(0, len(chords)-1)
chord_name = chords[chord_number][0]

chord = note_name + chord_name

f_root = noteToFreq(fullToneName(note_name),3)
f_root *= 2 if f_root < 120 else 1

# create a way to play it
chordSample = createChordWave(f_root,chords[chord_number][1],1.0)
noteSamples = createNoteWaves(f_root,chords[chord_number][1],0.5)

playFlag = False
def playClick(dummy):
    global playFlag, chordSample, noteSample,htmlChord
    if not playFlag:
        print(chords[chord_number][1])
        playFlag = True

    playWave(chordSample)
    time.sleep(0.25)
    for sample in noteSamples:
        playWave(sample)
        time.sleep(0.25)
    playWave(chordSample)

htmlChord = chord.replace('b','<sup>b</sup>').replace('#','<sup>#</sup>')

button = widgets.Button(description='Play')
button.on_click(playClick)
display(HTML(f'<h1>{htmlChord}</h1>'), button)

Button(description='Play', style=ButtonStyle())

['P1', 'm3', 'd5', 'bb7']


# Reference

<b>Triad practice</b> <i>[video](https://www.youtube.com/watch?v=xk9sLGu_KB0) 9:55-10:50, [video](https://www.youtube.com/watch?v=Lr_LLkozJ7Y)</i>

<b>Solfege practice</b> <i>[video](https://www.google.com/search?q=learning+solfeig&rlz=1C1CHBF_enUS912US912&oq=learning+solfeig&aqs=chrome..69i57j33i10i160l3.2841j0j7&sourceid=chrome&ie=UTF-8#fpstate=ive&vld=cid:982b4cc3,vid:DKHP-rNSESE)</i>
* Ascending maj7 chord notes, sing relative note before playing, the descending chord notes


<pre>
Major scale: 1 2 3 4 5 6 7 8'
  intervals:  W W H W W W H

Natural minor scale: 1 2 <sup>b</sup>3 4 5 <sup>b</sup>6 <sup>b</sup>7 8'
          intervals:  W H  W W H  W  W
</pre>

A *natural* minor scale can be built from the same set of notes as a major scale. The key of the natural minor is the 6<sup>th</sup> degree of the major scale.

### Pattern System

<pre>
Patterns along a string:
  X = 0 2 4
  Y = 0 1 3
  Z = 0 2 3

Patterns accross strings (root is on first note):
  1 Ionian     XX (YYZZ) = Major
  2 Dorian     ZX (XXYY)
  3 Phrygian   YZ (ZXXX)
  4 Lydian     XY (YZZX)
  5 Mixolydian XXX (YYZ)
  6 Aeolian    ZZ (XXXY) = Natural Minor
  7 Locrian    YY (ZZXX)

Rules across strings:
  1. Move up a fret from X to Y pattern
  2. Move up a fret from G string to B string

Rules along strings:
  1. Start next pattern on next fret from pattern X to X
  2. Start next pattern on 2nd fret after any other patterns
</pre>


### Pentatonic Shapes

<pre>
<b>Major</b> - R 2 3 5 6
╒═════╤══R══╤═════╤══2══╕   ╒═════╤══2══╤═════╤══3══╤═════╕   ╒══3══╤═════╤═════╤══5══╕
╞══3══╪═════╪═════╪══5══╡   ╞═════╪══5══╪═════╪══6══╪═════╡   ╞══6══╪═════╪═════╪══R══╡
╞══6══╪═════╪═════╪══R══╡   ╞═════╪══R══╪═════╪══2══╪═════╡   ╞══2══╪═════╪══3══╪═════╡
╞══2══╪═════╪══3══╪═════╡   ╞══3══╪═════╪═════╪══5══╪═════╡   ╞══5══╪═════╪══6══╪═════╡
├─────┼──5──┼─────┼──6──┤   ├─────┼──6──┼─────┼─────┼──R──┤   ├─────┼──R──┼─────┼──2──┤
└─────┴──R──┴─────┴──2──┘   └─────┴──2──┴─────┴──3──┴─────┘   └──3──┴─────┴─────┴──5──┘
╒═════╤══5══╤═════╤══6══╕   ╒══6══╤═════╤═════╤══R══╕
╞═════╪══R══╪═════╪══2══╡   ╞══2══╪═════╪══3══╪═════╡
╞══3══╪═════╪═════╪══5══╡   ╞══5══╪═════╪══6══╪═════╡
╞══6══╪═════╪═════╪══R══╡   ╞══R══╪═════╪══2══╪═════╡
├─────┼──2──┼─────┼──3──┤   ├──3──┼─────┼─────┼──5──┤
└─────┴──5──┴─────┴──6──┘   └──6──┴─────┴─────┴──R──┘

<b>Minor</b> - R b3 4 5 b7
╒══R══╤═════╤═════╤═b3══╕   ╒═════╤═b3══╤═════╤══4══╕   ╒═════╤══4══╤═════╤══5══╤═════╕
╞══4══╪═════╪══5══╪═════╡   ╞══5══╪═════╪═════╪═b7══╡   ╞═════╪═b7══╪═════╪══R══╪═════╡
╞═b7══╪═════╪══R══╪═════╡   ╞══R══╪═════╪═════╪═b3══╡   ╞═════╪═b3══╪═════╪══4══╪═════╡
╞═b3══╪═════╪══4══╪═════╡   ╞══4══╪═════╪══5══╪═════╡   ╞══5══╪═════╪═════╪═b7══╪═════╡
├──5──┼─────┼─────┼─b7──┤   ├─────┼─b7──┼─────┼──R──┤   ├─────┼──R──┼─────┼─────┼─b3──┤
└──R──┴─────┴─────┴─b3──┘   └─────┴─b3──┴─────┴──4──┘   └─────┴──4──┴─────┴──5──┴─────┘
╒══5══╤═════╤═════╤═b7══╕   ╒═════╤═b7══╤═════╤══R══╕
╞══R══╪═════╪═════╪═b3══╡   ╞═════╪═b3══╪═════╪══4══╡
╞══4══╪═════╪══5══╪═════╡   ╞══5══╪═════╪═════╪═b7══╡
╞═b7══╪═════╪══R══╪═════╡   ╞══R══╪═════╪═════╪═b3══╡
├─────┼─b3──┼─────┼──4──┤   ├─────┼──4──┼─────┼──5──┤
└──5──┴─────┴─────┴─b7──┘   └─────┴─b7──┴─────┴──R──┘
</pre>

### Chord Shapes

<pre>
      <b>1</b> (E)              <b>2</b> (C)              <b>3</b> (A)              <b>4</b> (F)              <b>5</b> (D)
╔══X══╦══╦══╤══X   X══╦══╦══╦══╤══X   X══╦══╦══╦══╤══X   X══X══╦══╦══╤══╕   X══X══╦══╦══╤══╕
R  ║  ║  ║  5  │   ║  ║  ║  5  │  │   ║  R  ║  ║  │  │   ║  ║  ║  ║  5──R   ║  ║  R  ║  │  │
╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤
║  ║  ║  3  │  │   ║  ║  ║  ║  R  │   ║  ║  ║  ║  │  │   ║  ║  ║  3  │  │   ║  ║  ║  ║  │  │
╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤
║  ║  R  ║  │  │   ║  ║  3  ║  │  │   ║  ║  5  R  3  │   ║  ║  R  ║  │  │   ║  ║  ║  5  │  3
╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤
║  ║  ║  ║  │  │   ║  R  ║  ║  │  │   ║  ║  ║  ║  │  │   ║  ║  ║  ║  │  │   ║  ║  ║  ║  R  │
╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤   ╟──╫──╫──╫──┼──┤